# Mine Safety

We're interested in [US mine safety](https://arlweb.msha.gov/drs/drshome.htm), thank goodness we can search for these things.

## Setup: Import what you'll need to search and scrape and Selenium

In [51]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

## Starting from `https://arlweb.msha.gov/drs/drshome.htm`, search for every operator with 'dirt' in their name, including abandoned mines.

> - *Tip: If you can't make an element work using name, class or ID, try to use the XPath*

In [52]:
driver = webdriver.Chrome()
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

In [21]:
#text_input = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[3]/td[1]/font/b')

In [53]:
text_input = driver.find_element_by_name('OperSearch')

In [54]:
driver.execute_script("arguments[0].scrollIntoView(true)", text_input)

In [55]:
text_input.send_keys('dirt')

In [56]:
button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table/tbody/tr[7]/td[3]/input[1]')
button.click()

In [73]:
contractors = driver.find_elements_by_tag_name('tr')
for contractor in contractors:
    print ('_______')
    print(contractor.text)

_______
Operator Name or Mine Name
Search  
_______
Abandoned*
Indicates Mine is Abandoned and Sealed
*CM (Coal or Metal Mine/Nonmetal Mine)
C
M ...... Coal
...... Metal/Nonmetal
_______
Abandoned*
_______
Indicates Mine is Abandoned and Sealed
_______
*CM (Coal or Metal Mine/Nonmetal Mine)
_______
C
M ...... Coal
...... Metal/Nonmetal
_______
ID State Operator Mine Name Type CM* Status Commodity More Info
_______
3503598
OR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC 
_______
1401575
KS  Bender Sand & Dirt   BENDER SAND & DIRT Surface M  Intermittent  Construction Sand and Gravel 
_______
5001797
AK  Dirt Company   Bush Pilot Surface M  Intermittent  Construction Sand and Gravel 
_______
2103723
MN  Dirt Doctor Inc   Rock Lake Plant Surface M  Intermittent  Construction Sand and Gravel 
_______
2103914
MN  Dirt Work Specialists LLC   Astec Plant Surface M  Intermittent  Construction Sand and Gravel 
_______
4104757
TX  Dirt Works   Portable 

## Scrape the results page, saving it as `dirt-operators.csv`

> - *Tip: Think about what each row in your dataset will be, and start by looping through that*
> - *Tip: Printing is cool and good! Print everything! Move it into a dictionary later.*
> - *Tip: If you don't want a row, think about what's in the row that makes it different. You can use an `if` statement or list slicing to skip the ones you aren't interested in.*
> - *Tip: Make sure your dictionary and your loop variable have DIFFERENT NAMES*
> - *Tip: After you've made your dictionary (and printed it, of course), you'll want to add it to your list of rows*
> - *Tip: Be sure to import pandas to convert it to a dataframe*
> - *Tip: Make sure you don't include the index when saving your dataframe*

### Hopefully you know that each `tr` is supposed to be a row of your data. What is the index of the first row element that is actually a result?

> - *Tip: `.text` will help you here.*
> - *Tip: You aren't interesting in annotations or anything, just mines and where they are from*
> - *Tip: Using `print("-----")` will help you keep track of different rows*
> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third. You can use this to skip ahead to the 'good' data if you want*

In [74]:
contractors = driver.find_elements_by_tag_name('tr')
for contractor in contractors:
    print ('_______')
    print(contractor.text)

_______
Operator Name or Mine Name
Search  
_______
Abandoned*
Indicates Mine is Abandoned and Sealed
*CM (Coal or Metal Mine/Nonmetal Mine)
C
M ...... Coal
...... Metal/Nonmetal
_______
Abandoned*
_______
Indicates Mine is Abandoned and Sealed
_______
*CM (Coal or Metal Mine/Nonmetal Mine)
_______
C
M ...... Coal
...... Metal/Nonmetal
_______
ID State Operator Mine Name Type CM* Status Commodity More Info
_______
3503598
OR  Newberg Rock & Dirt   Newberg Rock & Dirt Surface M  Active  Crushed, Broken Stone NEC 
_______
1401575
KS  Bender Sand & Dirt   BENDER SAND & DIRT Surface M  Intermittent  Construction Sand and Gravel 
_______
5001797
AK  Dirt Company   Bush Pilot Surface M  Intermittent  Construction Sand and Gravel 
_______
2103723
MN  Dirt Doctor Inc   Rock Lake Plant Surface M  Intermittent  Construction Sand and Gravel 
_______
2103914
MN  Dirt Work Specialists LLC   Astec Plant Surface M  Intermittent  Construction Sand and Gravel 
_______
4104757
TX  Dirt Works   Portable 

### Loop through each operator result, printing its name

> - *Tip: If you have a list called `animals`, `animals[2:]` will skip the first two and start with the third.*
> - *Tip: You can use list slicing or an `if` statement to skip the non-data row(s). List slicing is probably easier, even if you aren't comfortable with it.*
> - *Tip: or honestly you can use `try` and `except` if you know how it works.*
> - *Tip: Once you have the "right" rows of data, you're going to be looking for a certain tag inside*
> - *Tip: Sometimes you can't say "give me this class," and instead you have to say "give me all of the `div` elements, and then give me the third one."*

### Loop through each operator result, printing its ID

There should be ONE code per row, and NO empty rows between them.

In [108]:
contractors = driver.find_elements_by_tag_name('tr')
for contractor in contractors[7:-1]:
    items = contractor.find_elements_by_tag_name('td')
    print ('_______')
    print (items[0].text)

_______
3503598
_______
1401575
_______
5001797
_______
2103723
_______
2103914
_______
4104757
_______
0801306
_______
3901432
_______
3609624
_______
3609931
_______
1519799
_______
4407379
_______
4407296
_______
0203332
_______
2901986
_______
0801417
_______
4300768
_______
4300776
_______
2302283
_______
2103518


## Saving the results

### Loop through each `tr` to create a list of dictionaries

Each dictionary must contain

- Operator ID
- Operator name
- Mine name
- State
- Mine type
- Coal or metal
- Status
- Commodity

Create a new dictionary for each row.

> - *Tip: Start with an empty dictionary, then add the keys one at a time like we did during class*
> - *Tip: You might want to save all of the cells in a variable, then use indexes to get the second, third, fourth, etc.*
> - *Tip: I know you already skipped a bunch of rows already, but one of them still might be bad! Which one is it? How can you skip it? You might need to slice out some of the end of your list, too. Use `print` to help you debug, or just look at the page closely.*
> - *Tip: Or, if you did the other homework already, `try` / `except` is also an option*

In [132]:
rows = []

contractors = driver.find_elements_by_tag_name('tr')
for contractor in contractors[7:-1]:
    items1 = contractor.find_elements_by_tag_name('td')
    
    row = {}
    
    row['ID'] = items1[0].text
    row['State'] = items1[1].text
    row['Operator']=items1[2].text
    row['Mine Name']=items1[3].text
    row['Type']=items1[4].text
    row['CM']=items1[5].text
    row['Status']=items1[6].text
    row['Commodity']=items1[7].text

    rows.append(row)

In [130]:
rows

[{'ID': '3503598',
  'State': 'OR ',
  'Operator': 'Newberg Rock & Dirt  ',
  'Mine Name': 'Newberg Rock & Dirt',
  'Type': 'Surface',
  'CM': 'M ',
  'Status': 'Active ',
  'Commodity': 'Crushed, Broken Stone NEC '},
 {'ID': '1401575',
  'State': 'KS ',
  'Operator': 'Bender Sand & Dirt  ',
  'Mine Name': 'BENDER SAND & DIRT',
  'Type': 'Surface',
  'CM': 'M ',
  'Status': 'Intermittent ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '5001797',
  'State': 'AK ',
  'Operator': 'Dirt Company  ',
  'Mine Name': 'Bush Pilot',
  'Type': 'Surface',
  'CM': 'M ',
  'Status': 'Intermittent ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '2103723',
  'State': 'MN ',
  'Operator': 'Dirt Doctor Inc  ',
  'Mine Name': 'Rock Lake Plant',
  'Type': 'Surface',
  'CM': 'M ',
  'Status': 'Intermittent ',
  'Commodity': 'Construction Sand and Gravel '},
 {'ID': '2103914',
  'State': 'MN ',
  'Operator': 'Dirt Work Specialists LLC  ',
  'Mine Name': 'Astec Plant',
  'Type': 'Surf

### Save that to a CSV named `dirt-operators.csv`

In [133]:
import pandas as pd

df = pd.DataFrame(rows)

,CM,Commodity,ID,Mine Name,Operator,State,Status,Type
0,M,"Crushed, Broken Stone NEC",3503598,Newberg Rock & Dirt,Newberg Rock & Dirt,OR,Active,Surface
1,M,Construction Sand and Gravel,1401575,BENDER SAND & DIRT,Bender Sand & Dirt,KS,Intermittent,Surface
2,M,Construction Sand and Gravel,5001797,Bush Pilot,Dirt Company,AK,Intermittent,Surface
3,M,Construction Sand and Gravel,2103723,Rock Lake Plant,Dirt Doctor Inc,MN,Intermittent,Surface
4,M,Construction Sand and Gravel,2103914,Astec Plant,Dirt Work Specialists LLC,MN,Intermittent,Surface
5,M,Construction Sand and Gravel,4104757,Portable #1,Dirt Works,TX,Intermittent,Surface
6,M,"Sand, Common",0801306,River Road Pit,"Holley Dirt Company, Inc",FL,Active,Surface
7,M,Construction Sand and Gravel,3901432,PORTABLE SCREENER,Krueger Brothers Gravel & Dirt,SD,Intermittent,Surface
8,M,Construction Sand and Gravel,3609624,Forbes Pit,M R Dirt,PA,Temporarily Idled,Surface
9,M,Dimension Stone NEC,3609931,Camptown Quarry,M.R. Dirt Inc.,PA,Intermittent,Surface


In [135]:
df.to_csv("scraping-mines.csv")

### Open the CSV file and examine the first few.

Make sure you didn't save that extra weird unnamed index column.